# ФИНАЛЬНОЕ ЗАДАНИЕ: предсказание победителя в онлайн-игре

# Автор: Аппалонов Артем

# Введение
Dota 2 — многопользовательская компьютерная игра жанра MOBA. Игроки играют между собой матчи. В каждом матче, как правило, участвует 10 человек. Матчи формируются из живой очереди, с учётом уровня игры всех игроков. Перед началом игры игроки автоматически разделяются на две команды по пять человек. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника, трон. Существуют разные режимы игры, мы будем рассматривать режим Captain's Mode, в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

Необходимо построить модель, которая по данным о первых пяти минутах матча будет предсказывать его исход — то есть определять команду-победителя.

# Библиотеки

В начале задания пропишем все библиотеки, которые могут потребоваться нам для решения задач.

In [3]:
import pandas as pd
import numpy as np
import time
import datetime
from random import sample
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Чтение файла с признаками и целями

Прочитаем файл с данными features.csv и разделим его на две части: признаковое описание и целевые переменные.

In [4]:
features = pd.read_csv('./features.csv', index_col='match_id')

Проверим выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. И занесем данный список в переменную С, а затем посчитаем количество пропущенных данных.

In [5]:
С = features.count()
lose = С[С < 97230]
print('Количество столбцов, в которых пропущены данные:','\n',len(lose))
print('Признаки, в которых имеются пропуски в данных:')
print(lose)

Количество столбцов, в которых пропущены данные: 
 12
Признаки, в которых имеются пропуски в данных:
first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64


Пропуски в столбцах first_blood_player1/player2 связаны с тем, что к событию "первая кровь" (то есть первое убийство персонажа игрока противника) могут быть причастны не все игроки, пропуски же в столбцах radiant_bottle_time\courier_time связаны с тем, что игроки не успели за первые 5 минут заработать достаточно донатов и купить эти предметы (bottle и courier).

Заменили пропуски на нули с помощью функции fillna():

In [6]:
features.fillna(0, inplace = True)

Проверим, что в столбцах больше нет пропусков:

In [7]:
С = features.count()
lose = С[С < 97230]
print('Количество столбцов, в которых пропущены данные:','\n',len(lose))

Количество столбцов, в которых пропущены данные: 
 0


Для того чтобы обучение проходило быстрее, возьмем только половину данных из нашей выборки в случайном порядке:

In [8]:
features_d = features.take(np.random.permutation(len(features))[:int((len(features))/2)])

Целевая переменная хранится в колонке "radiant_win" (значения в ней: 1, если победила команда Radiant, 0 — иначе), перепишем её как отдельный столбец, а затем удалим из исходных данных, также удалим все стлбцы, которые содержат в себе информацию, которая выходит за рамки 5 минут матча. 

Вектор ответов обозначим за T (target).

In [9]:
T = features_d['radiant_win'].values

In [10]:
features_d.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis='columns', inplace=True)

D:\ANACONDA\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Теперь признаки для обучения представлены в переменной features, а целевая переменная в переменной T. Таким образом, мы подготовили обучающую выборку и можем начинать применять к данным методы машинного обучения. Решение проведем в два этапа - с помощью градиентного бустинга и логистической регрессии.

# Подход 1: градиентный бустинг "в лоб"

Метод градиентного бустинга последовательно строит композицию алгоритмов, причем каждый следующий алгоритм выбирается так, чтобы исправлять ошибки уже имеющейся композиции. Обычно в качестве базовых алгоритмов используют деревья небольшой глубины, поскольку их достаточно легко строить, и при этом они дают нелинейные разделяющие поверхности.

Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируем генератор разбиений для кросс-валидации по 5 блокам (KFold), не забыв перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
parametrs = [10, 20, 30]
Q = []
time = []
for n in parametrs:
    array = 0
    tmp = 0
    clf = GradientBoostingClassifier(n_estimators=n, learning_rate=0.3)
    clf.fit(features_d, T)
    start_time = 0
    start_time = datetime.datetime.now()
    array = cross_val_score(estimator=clf, X=features_d, y=T, cv=kf, scoring='roc_auc')
    m = np.mean(array)
    tmp = datetime.datetime.now() - start_time
    time.append(tmp)
    Q.append(m)

In [12]:
print('Время кросс-валидации для градиентного бустинга с 30 деревьями:')
print(time[len(time) - 1])

Время кросс-валидации для градиентного бустинга с 30 деревьями:
0:02:17.046226


In [13]:
print('Качество обучения алгоритма градиентного бустинга с 30 деревьями по  метрике AUC-ROC:')
print(Q[len(Q) - 1])

Качество обучения алгоритма градиентного бустинга с 30 деревьями по  метрике AUC-ROC:
0.6993932333836826


Если посмотреть на значения качества алгоритма для 10, 20 деревьев и сравнить с качеством для алгоритма для 30 деревьев, то можно увидеть, что с увеличение качество улучшается все меньше и меньше. То есть в определнный момент наблюдается ассимптотический рост. Иначе говоря, обучать алгоритм более, чем с 30 деревьями будет иметь смысл только в том случае, если значения точности обучения важно вплоть до конкретного знака после запятой(сотая или тысячная доли). При этом будет наблюдаться увеличение времени обучения.
Чтобы ускорить обучение алгоритма при увеличении количества деревьев можно предложить менять параметр learning rate, меняющий шаг градиентного спуска(learning_rate) или глубину деревьев(max_depth).

# Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов.

Оценим качество логистической регрессии с L2-регуляризацией с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберем при этом лучший параметр регуляризации (C), помня при этом, что линейные алгоритмы чувствительны к масштабу признаков, то есть предварительно отмасштабируем признаки.

In [15]:
scale = StandardScaler()
feat_scl = scale.fit_transform(features_d)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
parametrs2 = [0.01, 0.1, 1, 10, 100, 10000]
Q2 = []
time2 = []
for с in parametrs2:
    array = 0
    tmp = 0
    clf2 = LogisticRegression(penalty='l2', C=с)
    clf2.fit(feat_scl, T)
    start_time = 0
    start_time = datetime.datetime.now()
    array = cross_val_score(estimator=clf2, X=feat_scl, y=T, cv=kf, scoring='roc_auc')
    m = np.mean(array)
    tmp = datetime.datetime.now() - start_time
    time2.append(tmp)
    Q2.append(m)

In [16]:
print(*Q2)

0.7153316839647503 0.7152412874794838 0.7152258310693542 0.7152244594052032 0.7152243831343252 0.7152242222746709


Наилучшее качество получилось у алгоритма логистической регрессии с параметром С = 0.01. Оно принимает значение 0.7165421785434388.

In [17]:
print('Время кросс-валидации для алгоритма с данным параметром:')
print(time2[0])

Время кросс-валидации для алгоритма с данным параметром:
0:00:02.919192


Заметим, что в данном случае качество логистической регрессии с масштабированными признаками на входе оказалось выше, чем качество градиентного бустинга (0.72 и 0.70 соответственно). Можно сказать, что алгоритмы дают одинаковое качество с точностью до второго знака после запятой, однако для алгоритма логистической регрессии необходимо провести тщательную предобработку данных. Это также говорит о достаточной степени независимости в наблюдаемых данных. При этом, время кросс-валидации логистической регресси гораздо меньше времени кросс-валидации градиентного бустинга (соответственно, время обучения также меньше).

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберем их из выборки, и проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.

In [18]:
features_d.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero' ,'d5_hero'],axis='columns', inplace=True)

D:\ANACONDA\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
scale2 = StandardScaler()
feat_scl2 = scale.fit_transform(features_d)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
Q2 = []
time2 = []
for c in parametrs2:
    array = 0
    tmp = 0
    clf2 = LogisticRegression(penalty='l2', C=c)
    clf2.fit(feat_scl, T)
    start_time = 0
    start_time = datetime.datetime.now()
    array = cross_val_score(estimator=clf2, X=feat_scl2, y=T, cv=kf, scoring='roc_auc')
    m = np.mean(array)
    tmp = datetime.datetime.now() - start_time
    time2.append(tmp)
    Q2.append(m)

In [20]:
print(*Q2)

0.7155367188626645 0.715459320222543 0.7154491098521729 0.7154456625293749 0.7154457644422798 0.7154457898078512


Наилучшее качество алгоритма опять было получено с параметром С = 0.01. Стоит правда отметить, что качество алгоритма после удаления категориальных признаков немного увеличилось. Несущественно (в четвертом знаке после запятой), но все же увеличилось. Скорее всего это произошло из-за того, что эти данные были избыточны и приводили к "путанице" алгоритма при обучении (происходила переподгонка).

На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясним из данных, сколько различных идентификаторов героев существует в данной игре (используем фукнцию unique).

In [22]:
N = np.array(features['r1_hero'])
H = np.unique(N)
print('Число различных идентификаторов: ', len(H))

Число различных идентификаторов:  108


Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Добавим полученные признаки к числовым, которые были использованы во втором пункте данного этапа.

In [23]:
hero_c = [c for c in features.columns if 'hero' in c]
all_heroes_id = np.unique(features[hero_c])
wb = {}
for id in all_heroes_id:
    r = [(features['r%d_hero' % n] == id) + 0 for n in range(1, 6)]
    d = [(features['d%d_hero' % n] == id) + 0 for n in range(1, 6)]
    wb['hero%s' % id] = sum(r) - sum(d)
X_pick = features.assign(**wb)

Теперь вновь сократим данные:

In [24]:
features_pick = X_pick.take(np.random.permutation(len(features))[:int((len(features))/2)])
T2 = features_pick['radiant_win'].values
features_pick.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis='columns', inplace=True)

D:\ANACONDA\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.

In [25]:
scale3 = StandardScaler()
feat_scl3 = scale.fit_transform(features_pick)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
Q3 = []
time2 = []
for c in parametrs2:
    array = 0
    tmp = 0
    clf3 = LogisticRegression(penalty='l2', C=c)
    clf3.fit(feat_scl3, T2)
    start_time = 0
    start_time = datetime.datetime.now()
    array = cross_val_score(estimator=clf3, X=feat_scl3, y=T2, cv=kf, scoring='roc_auc')
    m = np.mean(array)
    tmp = datetime.datetime.now() - start_time
    time2.append(tmp)
    Q3.append(m)

In [26]:
print(*Q3)

0.7463087295698905 0.7462360157287664 0.7462213960690445 0.7462201915633251 0.746220175061526 0.7462203356241887


Наилучшее качество алгоритма снова было получено с параметром С = 0.01 (Значение 0.75). Стоит также отметить, что качество алгоритма увеличилось почти на 0.4. Это говорит о том, что информация о героях является важным признаком и при правильной кодировке его можно использовать для повышения качества алгоритма.

In [27]:
clf3 = LogisticRegression(penalty='l2', C=0.01)
clf3.fit(feat_scl3, T2)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Тестирование алгоритма

Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью модели логистической регрессии (она оказалась лучшей с точки зрения AUC-ROC на кросс-валидации).

Сначала загрузим тестовую выборку и произведем её предобработку.

In [28]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')

In [29]:
features_test.fillna(0, inplace = True)
hero_c = [c for c in features_test.columns if 'hero' in c]
all_heroes_id = np.unique(features[hero_c])
wb = {}
for id in all_heroes_id:
    r = [(features_test['r%d_hero' % n] == id) + 0 for n in range(1, 6)]
    d = [(features_test['d%d_hero' % n] == id) + 0 for n in range(1, 6)]
    wb['hero%s' % id] = sum(r) - sum(d)
X_pick_test = features_test.assign(**wb)
feat_scl_test = scale.transform(X_pick_test)

Теперь произведем тестирование модели:

In [32]:
predict = clf3.predict_proba(feat_scl_test)
predict_win_of_Radiant = []
for now in predict:
    predict_win_of_Radiant.append(now[1])

Минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов

In [33]:
print('Минимальное значение прогноза: ', min(predict_win_of_Radiant))
print('Максимальное значение прогноза: ', max(predict_win_of_Radiant))

Минимальное значение прогноза:  0.010044047921850357
Максимальное значение прогноза:  0.9959494389036317


# ВЫВОД:

В данной работе проверялись два подхода для создания модели предсказания победителя оналйн-игры Dota 2. В результате, было выяснено, что модель логистической регрессии показала лучшее значение качества лучшей с точки зрения AUC-ROC на кросс-валидации, чем градиентный бустинг. Также процесс обучения и проверки качества занял гораздо меньше времени. Однако, стоит отметить, что линейная регрессия требовала гораздо более глубокой предобработки данных.

# P.S. Спасибо за то, что нашли время и проверили мою работу.